# Building a Basic CNN: The MNIST Dataset

In this notebook, we will build a simple CNN-based architecture to classify the 10 digits (0-9) of the MNIST dataset. The objective of this notebook is to become familiar with the process of building CNNs in Keras.

We will go through the following steps:
1. Importing libraries and the dataset
2. Data preparation: Train-test split, specifying the shape of the input data etc.
3. Building and understanding the CNN architecture 
4. Fitting and evaluating the model

Let's dive in.

# 1. Importing libraries and the dataset

Let's load the required libraries. From Keras, we need to import two main components:
1. `Sequential` from `keras.models`: `Sequential` is the keras abstraction for creating models with a stack of layers (MLP has multiple hidden layers, CNNs have convolutional layers, etc.). 
2. Various types of layers from `keras.layers`: These layers are added (one after the other) to the `Sequential` model

The keras `backend` is needed for keras to know that you are using tensorflow (not Theano) at the backend (the backend is <a href="https://keras.io/backend/">specified in a JSON file</a>). 


In [ ]:
# import warnings
import warnings as w
w.filterwarnings('ignore')

import pandas as pd , numpy as np 
import tensorflow 
from tensorflow.keras.datasets import mnist
from tensorflow import keras  
from keras import Sequential
from keras.layers import Dense , Dropout , Conv2D , Flatten , MaxPooling2D

In [ ]:
# import data 
mnist.load_data()[:5]

In [ ]:
# split the X_train , y_train , X_test , y_test
(X_train , y_train) , (X_test , y_test) = mnist.load_data()

In [ ]:
X_train[0]

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
# the orignal image like this 
plt.figure(figsize=(10,5))
plt.subplot(2,4,1)
plt.imshow(X_train[0])
plt.subplot(2,4,2)
plt.imshow(X_train[1])
plt.subplot(2,4,3)
plt.imshow(X_train[2])
plt.subplot(2,4,4)
plt.imshow(X_train[3])
plt.subplot(2,4,5)
plt.imshow(X_train[4])
plt.subplot(2,4,6)
plt.imshow(X_train[5])
plt.subplot(2,4,7)
plt.imshow(X_train[6])
plt.subplot(2,4,8)
plt.imshow(X_train[7])

In [ ]:
# y_test data
y_train[:7]

In [ ]:
# train_data 
print('Train_data')
print('X_train',X_train.shape)
print('y_train',y_train.shape)
print('\n')
print('Test_data')
print('X_test',X_test.shape)
print('y_test',y_test.shape)

So we have 60,000 training and 10,000 test images each of size 28 x 28. Note that the images are grayscale and thus are stored as 2D arrays.
Also, let's sample only 20k images for training (just to speed up the training a bit).

In [ ]:
# sample only 20k images for training
idx = np.random.randint(X_train.shape[0], size=20000) # sample 20k indices from 0-60,000
X_train = X_train[idx, :]
y_train = y_train[idx]
print(X_train.shape)
print(y_train.shape)

# 2. Data Preparation

Let's prepare the dataset for feeding to the network. We will do the following three main steps:<br>

#### 2.1 Reshape the Data
First, let's understand the shape in which the network expects the training data. 
Since we have 20,000 training samples each of size (28, 28, 1), the training data (`x_train`) needs to be of the shape `(20000, 28, 28, 1)`. If the images were coloured, the shape would have been `(20000, 28, 28, 3)`.

Further, each of the 20,000 images have a 0-9 label, so `y_train` needs to be of the shape `(20000, 10)` where each image's label is represented as a 10-d **one-hot encoded vector**.

The shapes of `x_test` and `y_test` will be the same as that of `x_train` and `y_train` respectively.

#### 2.2 Rescaling (Normalisation)
The value of each pixel is between 0-255, so we will **rescale each pixel** by dividing by 255 so that the range becomes 0-1. Recollect <a href="https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn">why normalisation is important for training NNs</a>.

#### 2.3 Converting Input Data Type: Int to Float
The pixels are originally stored as type `int`, but it is advisable to feed the data as `float`. This is not really compulsory, but advisable. You can read <a href="https://datascience.stackexchange.com/questions/13636/neural-network-data-type-conversion-float-from-int">why conversion from int to float is helpful here</a>.


In [ ]:
# specify input dimensions of each image
img_rows , img_cols = (28,28)
input_shape = (img_rows , img_cols,1)

# batch_size , num_clasees , epochs

batch_size =128 
num_classes = 10 
epochs = 12

You Saw defference between batch , batch_size , epochs , iteration  :
[https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9](http://)

Let's now reshape the array x_train from shape (20000, 28, 28)to (20000, 28, 28, 1). Analogously for x_test.

In [ ]:
X_train[0]

In [ ]:
# reshape the X_train , X_test 
X_train = X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
X_test  = X_test.reshape(X_test.shape[0],img_rows,img_cols,1)
print('X_train_shape',X_train.shape)
print('X_test_shape',X_test.shape)

In [ ]:
X_train[0]

Now let's reshape y_train from (20000,) to (20000, 10). This can be conveniently done using the keras' `utils` module.

In [ ]:
# reshape the y_train , y_test # convert class labels (from digits) to one-hot encoded vectors

y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

print(y_train.shape)

Finally, let's convert the data type of x_train and x_test from int to float and normalise the images.

In [ ]:
# originally, the pixels are stored as ints
X_train.dtype

In [ ]:
# convert intp float 
X_train = X_train.astype('float')
X_test = X_test.astype('float')

In [ ]:
# normalize 
max_pixel_value = 255 
X_train = X_train/255 # X_train /= 255
X_test = X_test/255   # X_test /= 255

# 3. Building the Model

Let's now build the CNN architecture. For the MNIST dataset, we do not need to build a very sophisticated CNN - a simple shallow-ish CNN would suffice.

We will build a network with:

`two convolutional layers` having `32 and 64 filters respectively`, 

followed by a `max pooling layer`,

and then `Flatten` the output of the pooling layer to give us `a long vector`,

then add a `fully connected Dense layer with 128 neurons`, and finally

add a `softmax layer with 10 neurons`

The generic way to build a model in Keras is to instantiate a Sequential model and keep adding keras.layers to it. We will also use some dropouts.

In [ ]:
# model 
model = Sequential()

# a keras convolutional layer is called Conv2D
# help(Conv2D)
# note that the first layer needs to be told the input shape explicitly

# add first Conv2D layer
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))  # input_shape=(img_rows,img_colms,1)

# second Conv2D layer
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))

# add maxplloling
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten layer
model.add(Flatten())

# and fullyconected layer
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.5))

# Softmax layer
model.add(Dense(num_classes,activation='softmax')) # num_classes=10

# model summary 
model.summary()

#### Understanding Model Summary

It is a good practice to spend some time staring at the model summary above and verify the number of parameteres, output sizes etc. Let's do some calculations to verify that we understand the model deeply enough. 

- Layer-1 (Conv2D): We have used 32 kernels of size (3, 3), and each kernel has a single bias, so we have 32 x 3 x 3 (weights) + 32 (biases) = 320 parameters (all trainable). Note that the kernels have only one channel since the input images are 2D (grayscale). By default, a convolutional layer uses stride of 1 and no padding, so the output from this layer is of shape 26 x 26 x 32, as shown in the summary above (the first element `None` is for the batch size).

- Layer-2 (Conv2D): We have used 64 kernels of size (3, 3), but this time, each kernel has to convolve a tensor of size (26, 26, 32) from the previous layer. Thus, the kernels will also have 32 channels, and so the shape of each kernel is (3, 3, 32) (and we have 64 of them). So we have 64 x 3 x 3 x 32 (weights) + 64 (biases) = 18496 parameters (all trainable). The output shape is (24, 24, 64) since each kernel produces a (24, 24) feature map.

- Max pooling: The pooling layer gets the (24, 24, 64) input from the previous conv layer and produces a (12, 12, 64) output (the default pooling uses stride of 2). There are no trainable parameters in the pooling layer.

- The `Dropout` layer does not alter the output shape and has no trainable parameters.

- The `Flatten` layer simply takes in the (12, 12, 64) output from the previous layer and 'flattens' it into a vector of length 12 x 12 x 64 = 9216.

- The `Dense` layer is a plain fully connected layer with 128 neurons. It takes the 9216-dimensional output vector from the previous layer (layer l-1) as the input and has 128 x 9216 (weights) + 128 (biases) =  1179776 trainable parameters. The output of this layer is a 128-dimensional vector.

- The `Dropout` layer simply drops a few neurons.

- Finally, we have a `Dense` softmax layer with 10 neurons which takes the 128-dimensional vector from the previous layer as input. It has 128 x 10 (weights) + 10 (biases) = 1290 trainable parameters.

Thus, the total number of parameters are 1,199,882 all of which are trainable.

#  4. Fitting and Evaluating the Model

Let's now compile and train the model.

In [ ]:
# usual cross entropy loss
# choose any optimiser such as adam, rmsprop etc
# metric is accuracy

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [ ]:
# fit the model 
# batch_size =128 
# num_classes = 10 
# epochs = 12
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
# evaluate the model on test data
model.evaluate(X_test, y_test)

In [ ]:
print(model.metrics_names)

In [ ]:
model.evaluate(X_train, y_train)